In [171]:
import torch
import os
import json
import pandas as pd
import numpy as np

In [177]:
LABELS_DIR = f'../processed/3_final'

TENSOR_DIR = f'../tensors'

HERTZ = 100
WINDOW_LENGTH = 3 * HERTZ

ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING = {
    'water':0,
    'listerine':1,
}

NameError: name 'H' is not defined

In [173]:
def to_final(start_idx, end_idx, label, X):
    samples = np.empty((0, (WINDOW_LENGTH*3) + 1), dtype=X.dtype)  # Initialize samples with the correct shape and data type

    for i in range(round(start_idx+0.6), round(end_idx-WINDOW_LENGTH-0.6)):
        sample = X[i:i+WINDOW_LENGTH].T.flatten()
        sample = np.append(sample, label)
        samples = np.vstack((samples, sample))  # Add the sample as a new row

    #print(f'{X.shape} shape: {samples.shape} \n')
    return samples

In [174]:
def json_to_final(labels, acc):
    arr = np.array((0, (WINDOW_LENGTH*3)+1))
    print(f'{arr.shape}\n\n')

    for hand in labels:
        for action in labels[hand]:
            for bout in labels[hand][action]:
                index = acc[(acc.timestamp > bout['start']) & (acc.timestamp < bout['end'])].index
                if len(index) > 0:
                    start_idx = index[0]
                    end_idx = index[-1]
                    X = acc[['x','y','z']].values
                    rows = to_final(start_idx, end_idx, label=ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING[action], X=X)
                    if rows.size != 0:
                        print(arr.shape,'  ',rows.shape)
                        arr = np.vstack((arr, rows))


    return arr[:, :-1], arr[:, -1]


In [175]:
def json_to_tensor(json_file_path, acc_file_path):
    with open(json_file_path, 'r') as f:
        j_labels = json.load(f)

    acc = pd.read_csv(acc_file_path,skiprows=1)
    acc.timestamp = (acc.timestamp - acc.timestamp[0]) * 1e-9

    X, y = json_to_final(j_labels, acc, WINDOW_LENGTH)
    print(f'shape: {X.shape}  y: {y.shape}')

    return 5, 5




In [176]:
#open the labels from the processed file assumming you are in utils eg( ../processed/3_final/00/2024-01-06_14_29_18/labels.json )
for participant in sorted(os.listdir(LABELS_DIR)):
    if participant != '.DS_Store':
        sesions = []
        recordings = LABELS_DIR+f'/{participant}'
        for recording in sorted(os.listdir(recordings)):
            if recording != '.DS_Store':
                label_path = recordings+f'/{recording}/labels.json'
                acc_path = recordings+f'/{recording}/acceleration.csv'
                json_to_tensor(label_path, acc_path)
                #turn each one into a tensor
                #save each one the tensor dir


(2,)


(2,)    (457, 901)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 901

In [ ]:

acc = pd.read_csv('../processed/3_final/00/2024-01-06_14_29_18/acceleration.csv',skiprows=1)

In [ ]:
acc.timestamp = (acc.timestamp - acc.timestamp[0]) * 1e-9
acc

In [ ]:
X = acc[['x','y','z']].values


In [ ]:
X[0:15].T

In [ ]:
len(X.T.flatten())